In [36]:
import sys
import json

params = {}
default = {'training_folder': '../../scenarios/test', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/test', 'params': {}}


In [37]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal

In [38]:
base_folder = argv['training_folder']
pt_folder = base_folder + '/inputs/pt/'
road_folder = base_folder + '/inputs/road/'
input_folder = base_folder +'/inputs/'
od_folder = base_folder + '/inputs/od/'
output_folder = base_folder +'/outputs/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [39]:
from quetzal.engine.road_model import _reverse_geom
def split_quenedi_rlinks(rlinks, oneway='0'):
    if 'oneway' not in rlinks.columns:
        print('no column oneway. do not split')
        return
    links_r = rlinks[rlinks['oneway']==oneway].copy()
    if len(links_r) == 0:
        print('all oneway, nothing to split')
        return
    # apply _r features to the normal non r features
    r_cols = [col for col in links_r.columns if col.endswith('_r')]
    cols = [col[:-2] for col in r_cols]
    for col, r_col in zip(cols, r_cols):
        links_r[col] = links_r[r_col]
    # reindex with _r 
    links_r.index = links_r.index.astype(str) + '_r'
    # reverse links (a=>b, b=>a)
    links_r = links_r.rename(columns={'a': 'b', 'b': 'a'})
    links_r['geometry'] = links_r['geometry'].apply(lambda g: _reverse_geom(g))
    rlinks = pd.concat([rlinks, links_r])
    rlinks = rlinks.drop(columns=r_cols)
    return rlinks

In [40]:
rlinks = gpd.read_file(road_folder + 'road_links.geojson')
rlinks = rlinks.set_index('index')

In [41]:
rlinks = split_quenedi_rlinks(rlinks)

In [ ]:
rlinks['oneway'] = '1'

In [48]:
rlinks.to_file(road_folder + 'road_links.geojson',driver='GeoJSON') 
